# Target pack generator

In [ ]:
import os
import json
from mitreattack.stix20 import MitreAttackData
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# --- Load MITRE ATT&CK Data ---
display(Markdown("# MITRE ATT&CK Group Analysis"))
display(Markdown("## Loading ATT&CK Data"))

dataset_path = "data/enterprise-attack.json"
if not os.path.exists(dataset_path):
    display(Markdown("**Error: enterprise-attack.json not found. Please ensure the file is in the 'data' directory.**"))
else:
    mitre_attack_data = MitreAttackData(dataset_path)
    groups = mitre_attack_data.get_groups(remove_revoked_deprecated=True)
    display(Markdown(f"**Retrieved {len(groups)} ATT&CK groups.**"))

    # --- Group Selection ---
    display(Markdown("## Select a Group"))
    group_names = [group.name for group in groups]
    selected_group = widgets.Dropdown(
        options=group_names,
        description='Select Group:',
        layout=widgets.Layout(width='50%')
    )
    display(selected_group)

    output = widgets.Output()
    display(output)

    def display_group_info(group_name):
        """Displays information about the selected group."""
        with output:
            clear_output(wait=True)
            display(Markdown("---"))
            display(Markdown("## Group Information"))
            
            group = next((g for g in groups if g.name == group_name), None)
            if not group:
                display(Markdown("**Selected group not found.**"))
                return
            
            display(Markdown(f"### {group.name}"))
            display(Markdown(group.description))
            
            techniques_used_by_group = mitre_attack_data.get_techniques_used_by_group(group.id)
            display(Markdown(f"**Techniques Used: {len(techniques_used_by_group)}**"))
            
            mitre_techniques_used_by_group = []
            technique_list = ""
            for t in techniques_used_by_group:
                if t["object"]:
                    technique = t["object"]
                    attack_id = mitre_attack_data.get_attack_id(technique.id)
                    technique_list += f"- {technique.name} ({attack_id})\n"
                    mitre_techniques_used_by_group.append(attack_id)
            
            display(Markdown("### Techniques Used by Group"))
            display(Markdown(f"```\n{technique_list}\n```"))
            
            # --- Search for Related Analysis and Summaries ---
            display(Markdown("## Searching for Related Data"))
            
            analysis_folder = "MITRE_ATT&CK_Analysis"
            summary_folder = "MITRE_ATT&CK_Summaries"
            
            related_analysis = []
            related_summaries = []
            
            for root, _, files in os.walk(analysis_folder):
                for file in files:
                    if file.endswith(".json"):
                        file_path = os.path.join(root, file)
                        with open(file_path, "r", encoding="utf-8") as f:
                            data = json.load(f)
                            if data.get("id") in mitre_techniques_used_by_group:
                                related_analysis.append(file_path)
            
            for root, _, files in os.walk(summary_folder):
                for file in files:
                    if file.endswith(".md"):
                        file_path = os.path.join(root, file)
                        for attack_id in mitre_techniques_used_by_group:
                            if attack_id in file:
                                related_summaries.append(file_path)
                                break
            
            display(Markdown("### Related Analysis Files"))
            if related_analysis:
                display(Markdown("\n".join([f"- `{a}`" for a in related_analysis])))
            else:
                display(Markdown("**No related analysis files found.**"))
            
            display(Markdown("### Related Summary Files"))
            if related_summaries:
                display(Markdown("\n".join([f"- `{s}`" for s in related_summaries])))
            else:
                display(Markdown("**No related summary files found.**"))
            
            display(Markdown("---"))

    widgets.interactive(display_group_info, group_name=selected_group)


Retrieved 159 ATT&CK groups.


interactive(children=(Dropdown(description='Select Group:', layout=Layout(width='50%'), options=('Indrik Spide…